phase1

In [11]:
# Phase 1: Load data and extract vehicle plates based on annotations
# Matches each image with its same-named .xml annotation file, ignores <filename> inside XML
# Dependencies: opencv-python, xml.etree.ElementTree, os

import os
import cv2
import xml.etree.ElementTree as ET

# Paths (update these to your directories)
ANNOTATIONS_DIR = r"Plates2/Vehicle Plates annotations/Vehicle Plates annotations"      # folder containing .xml files
IMAGES_DIR      = r"Plates2/Vehicle Plates 1280x1280/Vehicle Plates 1280x1280"  # folder containing original images
OUTPUT_DIR      = r"plates33"   # where to save cropped plates

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Iterate through image files and process corresponding annotation
for img_file in os.listdir(IMAGES_DIR):
    if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue
    base_name, _ = os.path.splitext(img_file)

    img_path = os.path.join(IMAGES_DIR, img_file)
    xml_path = os.path.join(ANNOTATIONS_DIR, f"{base_name}.xml")
    if not os.path.isfile(xml_path):
        print(f"Skipping {img_file}: no annotation {base_name}.xml found.")
        continue

    # Load image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: could not read image {img_path}")
        continue
    actual_h, actual_w = img.shape[:2]

    # Parse XML annotation
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Get annotated image dimensions for scaling
    size = root.find('size')
    ann_w = float(size.findtext('width'))
    ann_h = float(size.findtext('height'))
    scale_x = actual_w / ann_w
    scale_y = actual_h / ann_h

    # Process one or multiple objects
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = float(bbox.findtext('xmin'))
        ymin = float(bbox.findtext('ymin'))
        xmax = float(bbox.findtext('xmax'))
        ymax = float(bbox.findtext('ymax'))

        # Scale to actual image coords
        x1 = int(xmin * scale_x)
        y1 = int(ymin * scale_y)
        x2 = int(xmax * scale_x)
        y2 = int(ymax * scale_y)

        # Crop and save
        plate = img[y1:y2, x1:x2]
        out_name = f"{base_name}_plate.png"
        cv2.imwrite(os.path.join(OUTPUT_DIR, out_name), plate)
        print(f"Extracted {out_name}")

print("Phase 1 extraction complete.")


Extracted 1_plate.png
Extracted 10_plate.png
Extracted 100_plate.png
Extracted 101_plate.png
Extracted 102_plate.png
Extracted 103_plate.png
Extracted 104_plate.png
Extracted 105_plate.png
Extracted 106_plate.png
Extracted 107_plate.png
Extracted 108_plate.png
Extracted 109_plate.png
Extracted 11_plate.png
Extracted 110_plate.png
Extracted 111_plate.png
Extracted 112_plate.png
Extracted 113_plate.png
Extracted 114_plate.png
Extracted 115_plate.png
Extracted 116_plate.png
Extracted 117_plate.png
Extracted 118_plate.png
Extracted 119_plate.png
Extracted 12_plate.png
Extracted 120_plate.png
Extracted 121_plate.png
Extracted 122_plate.png
Extracted 123_plate.png
Extracted 124_plate.png
Extracted 125_plate.png
Extracted 126_plate.png
Extracted 127_plate.png
Extracted 128_plate.png
Extracted 129_plate.png
Extracted 13_plate.png
Extracted 130_plate.png
Extracted 131_plate.png
Extracted 132_plate.png
Extracted 133_plate.png
Extracted 134_plate.png
Extracted 135_plate.png
Extracted 136_plate.pn

phase2

In [ ]:
import os
import cv2
import numpy as np

# Paths
PLATES_DIR = r"plates33"
OUTPUT_DIR = r"characters9"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def preprocess_plate(plate_img):
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        angles = [np.degrees(np.arctan2(y2-y1, x2-x1)) for [[x1,y1,x2,y2]] in lines if x2!=x1]
        if angles:
            m = np.median(angles)
            if abs(m) < 45:
                h, w = gray.shape
                M = cv2.getRotationMatrix2D((w//2, h//2), m, 1)
                gray = cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kern = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    # connect diacritics to base character
    binary = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kern, iterations=1)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kern, iterations=1)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kern, iterations=1)
    return binary


def extract_characters(binary_img, plate_name):
    num, labels, stats, cents = cv2.connectedComponentsWithStats(binary_img, 8)
    h, w = binary_img.shape
    area_tot = h * w
    comps = []
    for i in range(1, num):
        x, y, ww, hh, area = stats[i]
        ar = ww / float(hh) if hh > 0 else 0
        if area > area_tot * 0.001 and area < area_tot * 0.25 and 0.3 < ar < 3.5:
            cy = cents[i][1]
            comps.append((i, x, y, ww, hh, cy))
    if not comps:
        return []
    # line alignment
    ys = np.array([c[5] for c in comps])
    med = np.median(ys)
    tol = h * 0.15
    main = [c for c in comps if abs(c[5] - med) <= tol]
    main.sort(key=lambda c: c[1])  # left-to-right

    plate_dir = os.path.join(OUTPUT_DIR, plate_name)
    os.makedirs(plate_dir, exist_ok=True)
    saved = []
    for idx, (lid, x, y, ww, hh, cy) in enumerate(main):
        # extract component mask
        mask = (labels == lid).astype(np.uint8) * 255
        roi = mask[y:y+hh, x:x+ww]
        pad = 5
        pad_img = cv2.copyMakeBorder(roi, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=0)
        # resize preserving aspect
        tgt = (28, 28)
        h2, w2 = pad_img.shape
        ar = w2 / float(h2)
        if ar > 1:
            new_w = tgt[0]
            new_h = int(new_w / ar)
        else:
            new_h = tgt[1]
            new_w = int(new_h * ar)
        resized = cv2.resize(pad_img, (new_w, new_h))
        # invert so character is black, background white
        inv_char = 255 - resized
        # place on white canvas
        canvas = np.ones(tgt, dtype=np.uint8) * 255
        xo = (tgt[0] - new_w) // 2
        yo = (tgt[1] - new_h) // 2
        canvas[yo:yo+new_h, xo:xo+new_w] = inv_char
        # save
        out_path = os.path.join(plate_dir, f"char{idx}.png")
        cv2.imwrite(out_path, canvas)
        saved.append(canvas)
        print(f"Extracted character {idx} from {plate_name}")
    return saved

# Main loop
for f in os.listdir(PLATES_DIR):
    if not f.lower().endswith(('.png','.jpg','.jpeg')):
        continue
    img = cv2.imread(os.path.join(PLATES_DIR, f))
    if img is None:
        continue
    bin_img = preprocess_plate(img)
    extract_characters(bin_img, os.path.splitext(f)[0])
print("Done.")


phase 3

In [6]:
import os
import cv2
import numpy as np

# --- Configuration ---
INPUT_DIR   = r"alpha"             # original folder
OUTPUT_DIR  = r"alpha_processed"   # output folder with same subfolder structure
TARGET_SIZE = (28, 28)             # desired output image size

def preprocess_preserve_dark(img, tgt_size=TARGET_SIZE):
    # 1) Binarize: black glyph → white, background stays black
    _, bw = cv2.threshold(img, 250, 255, cv2.THRESH_BINARY_INV)

    # 2) Find all external contours (body + dots)
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        # Compute a bounding box that covers every contour
        x_min, y_min = img.shape[1], img.shape[0]
        x_max, y_max = 0, 0
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            x_max = max(x_max, x + w)
            y_max = max(y_max, y + h)
        cropped = img[y_min:y_max, x_min:x_max]
    else:
        cropped = img.copy()

    # 3) Resize to fit tgt_size, preserving aspect ratio
    h, w = cropped.shape
    scale = min(tgt_size[0] / w, tgt_size[1] / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # 4) Center on white canvas
    canvas = np.ones(tgt_size, dtype=np.uint8) * 255
    x_off = (tgt_size[0] - new_w) // 2
    y_off = (tgt_size[1] - new_h) // 2
    canvas[y_off:y_off+new_h, x_off:x_off+new_w] = resized

    return canvas

# --- Prepare output folders ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
for label in os.listdir(INPUT_DIR):
    src_folder = os.path.join(INPUT_DIR, label)
    dst_folder = os.path.join(OUTPUT_DIR, label)
    if not os.path.isdir(src_folder):
        continue
    os.makedirs(dst_folder, exist_ok=True)

    # Process each image in this label-folder
    for fname in os.listdir(src_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        in_path  = os.path.join(src_folder, fname)
        out_path = os.path.join(dst_folder, fname)

        img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️  Could not read {in_path}, skipping.")
            continue

        processed = preprocess_preserve_dark(img)
        cv2.imwrite(out_path, processed)

print("✅ All images preprocessed and saved to:", OUTPUT_DIR)


✅ All images preprocessed and saved to: alpha_processed


In [7]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Paths
data_dir   = r"alpha_processed"  # preprocessed 28×28 images, one subfolder per label
output_dir = r"models_flat"
os.makedirs(output_dir, exist_ok=True)

# 1. Load flattened data with optional simple augmentation
def load_data_flat(dir_path, augment=True):
    X, y = [], []

    def augment_image(img):
        # you can add flips/translations if desired
        return [img]

    for label in sorted(os.listdir(dir_path)):
        lblp = os.path.join(dir_path, label)
        if not os.path.isdir(lblp):
            continue

        for fname in os.listdir(lblp):
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            path = os.path.join(lblp, fname)
            img  = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None or img.shape != (28,28):
                continue

            variants = augment_image(img) if augment else [img]
            for im in variants:
                X.append(im.flatten())     # **flatten the 28×28 image**
                y.append(label)

    return np.array(X, dtype=np.float32), np.array(y)

print("Loading flattened data…")
X, y = load_data_flat(data_dir, augment=False)
print(f"Samples: {len(y)}, feature dim: {X.shape[1]}")  # should be 784

# 2. Encode & split
le    = LabelEncoder()
y_enc = le.fit_transform(y)
joblib.dump(le, os.path.join(output_dir, 'label_encoder.pkl'))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

# 3–4. Build pipelines with PCA + classifiers
models = {
    'DecisionTree': (DecisionTreeClassifier(random_state=42),
                     {'clf__max_depth': [10,20,None],
                      'clf__min_samples_leaf': [1,2,5]}),
    'SVM': (SVC(kernel='rbf', probability=True, random_state=42),
            {'clf__C': [0.1,1,10],
             'clf__gamma': ['scale','auto']}),
    'RandomForest': (RandomForestClassifier(n_estimators=200, random_state=42),
                     {'clf__max_depth': [10,20,None],
                      'clf__min_samples_leaf': [1,2,5]})
}

results = {}
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)

for name, (est, params) in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(whiten=True, random_state=42)),
        ('clf', est)
    ])
    # allow PCA variance tuning
    params['pca__n_components'] = [0.95, 0.98, 0.99]

    grid = GridSearchCV(pipe, params, cv=cv, n_jobs=-1, verbose=1)
    print(f"\nTraining {name}…")
    grid.fit(X_train, y_train)
    results[name] = grid

    # save best estimator
    joblib.dump(grid.best_estimator_,
                os.path.join(output_dir, f"{name.lower()}.pkl"))
    print(f"{name} best params:", grid.best_params_)

# 5. Evaluation
for name, grid in results.items():
    y_pred = grid.predict(X_test)
    acc    = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel('Predicted'); plt.ylabel('True')
    plt.savefig(os.path.join(output_dir, f"heatmap_{name}.png"))
    plt.close()

print("Phase 3 (flat features) complete—all models trained on flattened 28×28 data.")


Loading flattened data…
Samples: 4299, feature dim: 784

Training DecisionTree…
Fitting 10 folds for each of 27 candidates, totalling 270 fits
DecisionTree best params: {'clf__max_depth': 20, 'clf__min_samples_leaf': 1, 'pca__n_components': 0.95}

Training SVM…
Fitting 10 folds for each of 18 candidates, totalling 180 fits
SVM best params: {'clf__C': 10, 'clf__gamma': 'scale', 'pca__n_components': 0.95}

Training RandomForest…
Fitting 10 folds for each of 27 candidates, totalling 270 fits
RandomForest best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'pca__n_components': 0.95}

DecisionTree Accuracy: 0.7581
              precision    recall  f1-score   support

      1-alef       0.73      0.95      0.83        20
        10-d       0.56      0.45      0.50        20
      11-zal       0.69      0.55      0.61        20
        12-r       0.60      0.60      0.60        20
        13-z       0.61      0.70      0.65        20
       14-zh       0.72      0.65      0.68 

In [ ]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Paths
data_dir = r"alpha_processed"  # organized by label
output_dir = r"models_flat30"
os.makedirs(output_dir, exist_ok=True)

# 1. Load data with augmentation and resize to 28x28

def load_data(dir_path, augment=True):
    X, y = [], []
    # target size for HOG and pixel features
    win_size = (28, 28)
    # HOG parameters matching 28x28
    hog = cv2.HOGDescriptor(
        _winSize=win_size,
        _blockSize=(14,14),
        _blockStride=(7,7),
        _cellSize=(7,7),
        _nbins=9
    )
    def augment_image(img):
        rots = [0, -10, 10]
        out = []
        h, w = img.shape
        center = (w//2, h//2)
        for ang in rots:
            M = cv2.getRotationMatrix2D(center, ang, 1.0)
            rot = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            out.append(rot)
        return out

    for label in sorted(os.listdir(dir_path)):
        lbl_path = os.path.join(dir_path, label)
        if not os.path.isdir(lbl_path):
            continue
        for fname in os.listdir(lbl_path):
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            img = cv2.imread(os.path.join(lbl_path, fname), cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            # Resize to 28x28
            img = cv2.resize(img, win_size)
            imgs = augment_image(img) if augment else [img]
            for im in imgs:
                # extract HOG features
                desc = hog.compute(im)
                X.append(desc.flatten())
                y.append(label)
    return np.array(X), np.array(y)

print("Loading and augmenting data...")
X, y = load_data(data_dir)
print(f"Total samples: {len(y)}, feature dim: {X.shape[1]}")

# 2. Encode labels and split
y_encoder = LabelEncoder()
y_enc = y_encoder.fit_transform(y)
joblib.dump(y_encoder, os.path.join(output_dir, 'label_encoder.pkl'))
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

# 3. PCA for dimensionality reduction
pca = PCA(n_components=0.99, whiten=True, random_state=42)

# 4. Define models and hyperparameters
models = {
    'DecisionTree': (DecisionTreeClassifier(random_state=42),
                     {'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 5]}),
    'SVM': (SVC(kernel='rbf', probability=True, random_state=42),
            {'clf__C': [0.1, 1, 10], 'clf__gamma': ['scale', 'auto']})
}

results = {}
for name, (estimator, params) in models.items():
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca), ('clf', estimator)])
    grid = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
    print(f"Training {name}...")
    grid.fit(X_train, y_train)
    results[name] = grid
    joblib.dump(grid.best_estimator_, os.path.join(output_dir, f"{name.lower()}.pkl"))
    print(f"{name} best params: {grid.best_params_}")

# 5. Evaluation and accuracy reporting
for name, grid in results.items():
    y_pred = grid.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")

    # detailed report
    print(classification_report(y_test, y_pred, target_names=y_encoder.classes_))
    cm = confusion_matrix(y_test, y_pred)
    np.savetxt(os.path.join(output_dir, f"cm_{name}.csv"), cm, fmt='%d', delimiter=',')
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=y_encoder.classes_, yticklabels=y_encoder.classes_)
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel('Predicted'); plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"heatmap_{name}.png"))
    plt.close()

print("All done. Accuracies reported above.")


Loading and augmenting data...
Total samples: 12897, feature dim: 324
Training DecisionTree...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
DecisionTree best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 1}
Training SVM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
SVM best params: {'clf__C': 10, 'clf__gamma': 'scale'}
DecisionTree Accuracy: 0.8643
              precision    recall  f1-score   support

      1-alef       0.82      0.83      0.83        60
        10-d       0.84      0.82      0.83        60
      11-zal       0.86      0.80      0.83        60
        12-r       0.96      0.88      0.92        60
        13-z       0.78      0.78      0.78        60
       14-zh       0.81      0.78      0.80        60
      15-sin       0.72      0.78      0.75        60
     16-shin       0.78      0.70      0.74        60
      17-sad       0.71      0.85      0.77        60
      18-zad       0.76      0.80      0.78        60
   19-t-lo

In [38]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Paths
data_dir = r"alpha_processed"  # organized by label
output_dir = r"models_flat3"
os.makedirs(output_dir, exist_ok=True)

# Augmentation function
def augment_image(img):
    h, w = img.shape
    out = []

    # Rotation
    center = (w // 2, h // 2)
    for angle in [0, -10, 10]:
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(img, M, (w, h),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=255)
        out.append(rotated)

    # Zoom-out
    scale = 0.9
    nw, nh = int(w * scale), int(h * scale)
    if nw > 0 and nh > 0:
        resized = cv2.resize(img, (nw, nh))
        canvas = np.ones((h, w), dtype=np.uint8) * 255
        x0 = (w - nw) // 2
        y0 = (h - nh) // 2
        canvas[y0:y0 + nh, x0:x0 + nw] = resized
        out.append(canvas)

    # Gaussian noise
    noise = np.random.normal(0, 10, img.shape).astype(np.int16)
    noisy = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    out.append(noisy)

    # Contrast
    contrast = cv2.convertScaleAbs(img, alpha=1.2, beta=0)
    out.append(contrast)

    return out

# Data loader
def load_data(dir_path, augment=True):
    X, y = [], []
    win_size = (28, 28)
    hog = cv2.HOGDescriptor(
        _winSize=win_size,
        _blockSize=(14,14),
        _blockStride=(7,7),
        _cellSize=(7,7),
        _nbins=9
    )
    for label in sorted(os.listdir(dir_path)):
        lbl_path = os.path.join(dir_path, label)
        if not os.path.isdir(lbl_path):
            continue
        for fname in os.listdir(lbl_path):
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            raw = cv2.imread(os.path.join(lbl_path, fname), cv2.IMREAD_GRAYSCALE)
            if raw is None:
                continue
            img = cv2.resize(raw, win_size)
            variants = augment_image(img) if augment else [img]
            for im in variants:
                X.append(hog.compute(im).flatten())
                y.append(label)
    return np.array(X), np.array(y)

# Load data
print("Loading & augmenting data…")
X, y = load_data(data_dir, augment=True)
print(f"Samples: {len(y)}, feature dim: {X.shape[1]}")

# Encode labels and split
y_encoder = LabelEncoder()
y_enc = y_encoder.fit_transform(y)
joblib.dump(y_encoder, os.path.join(output_dir, 'label_encoder.pkl'))
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

# PCA
pca = PCA(n_components=0.99, whiten=True, random_state=42)

# Models and hyperparameters
models = {
    'DecisionTree': (DecisionTreeClassifier(random_state=42),
                     {'clf__max_depth': [10, 20, None],
                      'clf__min_samples_leaf': [1, 2, 5]}),
    'SVM': (SVC(kernel='rbf', probability=True, random_state=42),
            {'clf__C': [0.1, 1, 10],
             'clf__gamma': ['scale', 'auto']})
}

# Train and tune
results = {}
for name, (estimator, params) in models.items():
    print(f"Training {name}...")
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca),
        ('clf', estimator)
    ])
    grid = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    results[name] = grid
    joblib.dump(grid.best_estimator_, os.path.join(output_dir, f"{name.lower()}.pkl"))
    print(f"{name} best params: {grid.best_params_}")

# Evaluation
for name, model in results.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred, target_names=y_encoder.classes_))
    
    cm = confusion_matrix(y_test, y_pred)
    np.savetxt(os.path.join(output_dir, f"cm_{name}.csv"), cm, fmt='%d', delimiter=',')
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=y_encoder.classes_,
                yticklabels=y_encoder.classes_)
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"heatmap_{name}.png"))
    plt.close()

print("\nAll models trained and evaluated.")


Loading & augmenting data…
Samples: 25794, feature dim: 324
Training DecisionTree...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
DecisionTree best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 1}
Training SVM...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
SVM best params: {'clf__C': 10, 'clf__gamma': 'scale'}

DecisionTree Accuracy: 0.8946
              precision    recall  f1-score   support

      1-alef       0.93      0.93      0.93       120
        10-d       0.88      0.93      0.91       120
      11-zal       0.89      0.91      0.90       120
        12-r       0.89      0.82      0.86       120
        13-z       0.75      0.76      0.75       120
       14-zh       0.78      0.84      0.81       120
      15-sin       0.82      0.90      0.86       120
     16-shin       0.83      0.81      0.82       120
      17-sad       0.86      0.87      0.86       120
      18-zad       0.80      0.78      0.79       120
   19-t-long       

In [49]:
import os
import cv2
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Paths
data_dir = r"alpha_processed"  # organized by label
output_dir = r"models_flat30"
os.makedirs(output_dir, exist_ok=True)

# 1. Load data with augmentation and resize to 28x28
def load_data(dir_path, augment=True):
    X, y = [], []
    win_size = (28, 28)  # target size for HOG and pixel features
    hog = cv2.HOGDescriptor(
        _winSize=win_size,
        _blockSize=(14, 14),
        _blockStride=(7, 7),
        _cellSize=(7, 7),
        _nbins=9
    )
    
    def augment_image(img):
        rots = [0, -10, 10]  # rotations in degrees
        trans = [(-2, 0), (2, 0)]  # horizontal translations: left and right by 2 pixels
        out = []
        h, w = img.shape
        center = (w // 2, h // 2)
        
        # Rotations
        for ang in rots:
            M_rot = cv2.getRotationMatrix2D(center, ang, 1.0)
            rot_img = cv2.warpAffine(img, M_rot, (w, h), flags=cv2.INTER_LINEAR,
                                     borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            out.append(rot_img)
        
        # Translations on original image
        for dx, dy in trans:
            M_trans = np.float32([[1, 0, dx], [0, 1, dy]])
            trans_img = cv2.warpAffine(img, M_trans, (w, h), flags=cv2.INTER_LINEAR,
                                       borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            out.append(trans_img)
        
        return out

    for label in sorted(os.listdir(dir_path)):
        lbl_path = os.path.join(dir_path, label)
        if not os.path.isdir(lbl_path):
            continue
        for fname in os.listdir(lbl_path):
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            img = cv2.imread(os.path.join(lbl_path, fname), cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            # Resize to 28x28
            img = cv2.resize(img, win_size)
            imgs = augment_image(img) if augment else [img]
            for im in imgs:
                # Extract HOG features
                desc = hog.compute(im)
                X.append(desc.flatten())
                y.append(label)
    return np.array(X), np.array(y)

print("Loading and augmenting data...")
X, y = load_data(data_dir)
print(f"Total samples: {len(y)}, feature dim: {X.shape[1]}")

# 2. Encode labels and split
y_encoder = LabelEncoder()
y_enc = y_encoder.fit_transform(y)
joblib.dump(y_encoder, os.path.join(output_dir, 'label_encoder.pkl'))
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)

# 3. PCA for dimensionality reduction (without specifying n_components yet)
pca = PCA(whiten=True, random_state=42)

# 4. Define models and hyperparameters
models = {
    'DecisionTree': (DecisionTreeClassifier(random_state=42),
                     {'pca__n_components': [0.95, 0.99],
                      'clf__criterion': ['gini', 'entropy'],
                      'clf__max_depth': [10, 20, None],
                      'clf__min_samples_leaf': [1, 2, 5]}),
    'SVM': (SVC(kernel='rbf', probability=True, random_state=42),
            [{'pca__n_components': [0.95, 0.99],
              'clf__kernel': ['rbf'],
              'clf__C': [0.1, 1, 10],
              'clf__gamma': ['scale', 'auto']},
             {'pca__n_components': [0.95, 0.99],
              'clf__kernel': ['linear'],
              'clf__C': [0.1, 1, 10]}])
}

results = {}
for name, (estimator, params) in models.items():
    pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca), ('clf', estimator)])
    grid = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
    print(f"Training {name}...")
    grid.fit(X_train, y_train)
    results[name] = grid
    joblib.dump(grid.best_estimator_, os.path.join(output_dir, f"{name.lower()}.pkl"))
    print(f"{name} best params: {grid.best_params_}")

# 5. Evaluation and accuracy reporting
for name, grid in results.items():
    y_pred = grid.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")

    # Detailed report
    print(classification_report(y_test, y_pred, target_names=y_encoder.classes_))
    cm = confusion_matrix(y_test, y_pred)
    np.savetxt(os.path.join(output_dir, f"cm_{name}.csv"), cm, fmt='%d', delimiter=',')
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=y_encoder.classes_, yticklabels=y_encoder.classes_)
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel('Predicted'); plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"heatmap_{name}.png"))
    plt.close()

print("All done. Accuracies reported above.")

Loading and augmenting data...
Total samples: 21495, feature dim: 324
Training DecisionTree...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
DecisionTree best params: {'clf__criterion': 'entropy', 'clf__max_depth': 20, 'clf__min_samples_leaf': 1, 'pca__n_components': 0.95}
Training SVM...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
SVM best params: {'clf__C': 10, 'clf__gamma': 'scale', 'clf__kernel': 'rbf', 'pca__n_components': 0.95}
DecisionTree Accuracy: 0.8930
              precision    recall  f1-score   support

      1-alef       0.94      0.88      0.91       100
        10-d       0.89      0.85      0.87       100
      11-zal       0.89      0.90      0.90       100
        12-r       0.84      0.93      0.88       100
        13-z       0.77      0.67      0.72       100
       14-zh       0.79      0.83      0.81       100
      15-sin       0.85      0.85      0.85       100
     16-shin       0.78      0.76      0.77       100
      17-sad

phase 4


In [52]:
import os
import cv2
import numpy as np
import joblib

# Paths
dirs = {
    'plate': r'plates33',
    'chars': r'characters9',
    'models': r'models_flat30',
    'output': r'plate_results'
}
for d in dirs.values(): os.makedirs(d, exist_ok=True)

# Load models and encoder
dt = joblib.load(os.path.join(dirs['models'], 'decisiontree.pkl'))
svm = joblib.load(os.path.join(dirs['models'], 'svm.pkl'))
rf_path = os.path.join(dirs['models'], 'randomforest.pkl')
rf = joblib.load(rf_path) if os.path.isfile(rf_path) else None
le = joblib.load(os.path.join(dirs['models'], 'label_encoder.pkl'))

# HOG descriptor - MATCHED TO TRAINING PARAMETERS
hog = cv2.HOGDescriptor(
    _winSize=(28, 28),         # Match training size
    _blockSize=(14, 14),       # Match training
    _blockStride=(7, 7),       # Match training
    _cellSize=(7, 7),          # Match training
    _nbins=9
)

def compute_hog(img):
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Resize to 28x28 to match training
    img = cv2.resize(img, (28, 28))
    
    # Apply same preprocessing as training (no explicit thresholding in training)
    # If you still want thresholding, uncomment next line
    # _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    
    return hog.compute(img).flatten().reshape(1,-1)

# Iterate plates
for pf in os.listdir(dirs['plate']):
    name, _ = os.path.splitext(pf)
    char_dir = os.path.join(dirs['chars'], name)
    if not os.path.isdir(char_dir): 
        print(f"No character directory for {name}")
        continue
    
    # sorted char files
    files = sorted([f for f in os.listdir(char_dir) if f.startswith('char')],
                   key=lambda x: int(''.join(filter(str.isdigit,x)) or 0))
    
    if not files:
        print(f"No character files found in {char_dir}")
        continue
        
    preds = {'dt':[], 'svm':[], 'rf':[], 'vote':[]}
    
    for f in files:
        path = os.path.join(char_dir, f)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to read {path}")
            for k in preds: preds[k].append('_')
            continue
            
        # Compute HOG features with matched parameters
        feat = compute_hog(img)
        
        try:
            # DT prediction
            dt_idx = dt.predict(feat)[0]
            dt_c = le.inverse_transform([dt_idx])[0]
            preds['dt'].append(dt_c)
            
            # SVM prediction
            svm_idx = svm.predict(feat)[0]
            svm_c = le.inverse_transform([svm_idx])[0]
            preds['svm'].append(svm_c)
            
            # RF prediction if available
            if rf:
                rf_idx = rf.predict(feat)[0]
                rf_c = le.inverse_transform([rf_idx])[0]
                preds['rf'].append(rf_c)
            else:
                preds['rf'].append('_')
                
            # Ensemble voting with probabilities
            probs = np.zeros((feat.shape[0], len(le.classes_)))
            probs += dt.predict_proba(feat)
            probs += svm.predict_proba(feat)
            if rf: probs += rf.predict_proba(feat)
            
            avg = probs / (3 if rf else 2)
            vidx = np.argmax(avg, axis=1)[0]
            vprob = avg[0, vidx]
            vote_c = le.inverse_transform([vidx])[0] if vprob >= 0.5 else '?'  # Lowered threshold slightly
            preds['vote'].append(vote_c)
            
        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            for k in preds: preds[k].append('?')
    
    # Create final output
    final = ''.join(preds['vote'])
    
    # Save results
    out = os.path.join(dirs['output'], f"{name}_result.txt")
    with open(out, 'w', encoding='utf-8') as f:
        f.write('DT:' + ','.join(preds['dt']) + '\n')
        f.write('SVM:' + ','.join(preds['svm']) + '\n')
        if rf: f.write('RF:' + ','.join(preds['rf']) + '\n')
        f.write('VOTE:' + ','.join(preds['vote']) + '\n')
        f.write('FINAL:' + final)
        
    print(f"Recognized {name}: {final}")
    
print('Done.')

Recognized 100_plate: 36-three28-m32-ye36-three13-z10-d28-m28-m3-p
Recognized 101_plate: 9-kh35-two6-jim40-seven9-kh6-jim
Recognized 102_plate: 35-two28-m30-v35-two28-m28-m19-t-long28-m
Recognized 103_plate: 22-ghyin28-m27-le6-jim28-m
Recognized 104_plate: 41-eight41-eight17-sad41-eight28-m36-three35-two28-m
Recognized 105_plate: 40-seven19-t-long28-m40-seven35-two35-two19-t-long40-seven
Recognized 106_plate: 42-nine35-two8-h33-zero28-m43-anewfive19-t-long19-t-long28-m
Recognized 107_plate: 35-two41-eight10-d41-eight43-anewfive35-two34-one40-seven
Recognized 108_plate: 40-seven40-seven23-f41-eight40-seven3-p34-one3-p40-seven
Recognized 109_plate: 35-two10-d29-n11-zal39-six35-two35-two34-one28-m
Recognized 10_plate: 28-m9-kh10-d6-jim13-z35-two13-z39-six
Recognized 110_plate: 28-m42-nine32-ye33-zero41-eight27-le42-nine34-one28-m
Recognized 111_plate: 36-three41-eight27-le34-one35-two43-anewfive34-one40-seven
Recognized 112_plate: 34-one19-t-long17-sad42-nine36-three22-ghyin28-m
Recognize

phase5

In [53]:
import os
import re
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Paths
GROUND_TRUTH_CSV = r"plate_labels.txt"     # 'name,plate' where plate is e.g. "6-jim,9-kh,…"
PREDICTION_DIR   = r"plate_results"        # contains <base>_plate_result.txt
OUTPUT_DIR       = r"evaluation_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def strip_prefix(token):
    return token.split('-', 1)[-1]

# 1. Load ground truth (strip numeric prefixes)
gt_map = {}
with open(GROUND_TRUTH_CSV, 'r', encoding='utf-8') as f:
    lines = [ln.strip() for ln in f if ln.strip()]
for ln in lines[1:]:
    parts = ln.split(',')
    if len(parts) < 2:
        continue
    name = parts[0].strip()
    tail = ','.join(parts[1:]).strip()
    raw_tokens = [t.strip() for t in tail.split(',') if t.strip()]
    gt_map[name] = [strip_prefix(t) for t in raw_tokens]

# 2. Iterate plates and compute metrics
plate_total = 0
plate_exact_matches = 0
plate_exact_matches_dt = 0
plate_exact_matches_svm = 0
sequence_scores = []       # per-plate % correct in sequence for FINAL
all_gt, all_pred = [], []  # For FINAL predictions
all_pred_dt = []           # For DT predictions
all_pred_svm = []          # For SVM predictions

for name, gt_labels in gt_map.items():
    base = name.split('_')[0]
    res_file = os.path.join(PREDICTION_DIR, f"{base}_plate_result.txt")
    if not os.path.isfile(res_file):
        print(f"Missing result for {name}")
        continue

    with open(res_file, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()]

    # Extract lines for FINAL, DT, and SVM
    final_line = next((ln for ln in lines if ln.startswith('FINAL:')), None)
    dt_line = next((ln for ln in lines if ln.startswith('DT:')), None)
    svm_line = next((ln for ln in lines if ln.startswith('SVM:')), None)

    if not final_line or not dt_line or not svm_line:
        print(f"Missing FINAL, DT, or SVM line in {res_file}")
        continue

    # Process FINAL predictions (original)
    raw_pred = re.findall(r'\d+-[^\d,]+', final_line)
    pred_labels = [strip_prefix(t) for t in raw_pred]

    # Process DT predictions
    raw_pred_dt = re.findall(r'\d+-[^\d,]+', dt_line)
    pred_labels_dt = [strip_prefix(t) for t in raw_pred_dt]

    # Process SVM predictions
    raw_pred_svm = re.findall(r'\d+-[^\d,]+', svm_line)
    pred_labels_svm = [strip_prefix(t) for t in raw_pred_svm]

    L = min(len(gt_labels), len(pred_labels))
    if L == 0:
        print(f"Empty GT or prediction for {name}")
        continue

    gt_seq = gt_labels[:L]
    pred_seq = pred_labels[:L]
    pred_seq_dt = pred_labels_dt[:L]
    pred_seq_svm = pred_labels_svm[:L]

    # 2a) Plate-level exact match for FINAL
    plate_total += 1
    if gt_seq == pred_seq and len(gt_labels) == len(pred_labels):
        plate_exact_matches += 1

    # 2b) Plate-level exact match for DT
    if gt_seq == pred_seq_dt and len(gt_labels) == len(pred_labels_dt):
        plate_exact_matches_dt += 1

    # 2c) Plate-level exact match for SVM
    if gt_seq == pred_seq_svm and len(gt_labels) == len(pred_labels_svm):
        plate_exact_matches_svm += 1

    # 2d) Sequence percentage for FINAL (original)
    correct_chars = sum(1 for i in range(L) if gt_seq[i] == pred_seq[i])
    seq_pct = correct_chars / len(gt_labels) * 100
    sequence_scores.append(seq_pct)

    # Accumulate for token-level metrics
    all_gt.extend(gt_seq)
    all_pred.extend(pred_seq)
    all_pred_dt.extend(pred_seq_dt)
    all_pred_svm.extend(pred_seq_svm)

    print(f"Plate {name}: exact_match={gt_seq==pred_seq}, sequence_pct={seq_pct:.1f}%")

# 3. Aggregate metrics
plate_exact_acc = plate_exact_matches / plate_total * 100 if plate_total else 0
plate_exact_acc_dt = plate_exact_matches_dt / plate_total * 100 if plate_total else 0
plate_exact_acc_svm = plate_exact_matches_svm / plate_total * 100 if plate_total else 0
avg_sequence_acc = np.mean(sequence_scores) if sequence_scores else 0
token_acc = accuracy_score(all_gt, all_pred) * 100 if all_gt else 0
token_acc_dt = accuracy_score(all_gt, all_pred_dt) * 100 if all_gt else 0
token_acc_svm = accuracy_score(all_gt, all_pred_svm) * 100 if all_gt else 0

# 4. Save & print
with open(os.path.join(OUTPUT_DIR, 'metrics.txt'), 'w', encoding='utf-8') as f:
    f.write(f"Plates processed: {plate_total}\n")
    f.write(f"Plate exact-match accuracy (FINAL): {plate_exact_acc:.2f}%\n")
    f.write(f"Plate exact-match accuracy (DT):    {plate_exact_acc_dt:.2f}%\n")
    f.write(f"Plate exact-match accuracy (SVM):   {plate_exact_acc_svm:.2f}%\n")
    f.write(f"Average sequence accuracy (FINAL):  {avg_sequence_acc:.2f}%\n")
    f.write(f"Token-level accuracy (FINAL):       {token_acc:.2f}%\n")
    f.write(f"Token-level accuracy (DT):          {token_acc_dt:.2f}%\n")
    f.write(f"Token-level accuracy (SVM):         {token_acc_svm:.2f}%\n\n")
    if all_gt:
        f.write("Classification Report (token-level, FINAL):\n")
        f.write(classification_report(all_gt, all_pred, zero_division=0))

print("\nPhase 5 evaluation complete.")
print(f"Plate exact-match (FINAL): {plate_exact_acc:.2f}%")
print(f"Plate exact-match (DT):    {plate_exact_acc_dt:.2f}%")
print(f"Plate exact-match (SVM):   {plate_exact_acc_svm:.2f}%")
print(f"Avg sequence acc (FINAL):  {avg_sequence_acc:.2f}%")
print(f"Token-level acc (FINAL):   {token_acc:.2f}%")
print(f"Token-level acc (DT):      {token_acc_dt:.2f}%")
print(f"Token-level acc (SVM):     {token_acc_svm:.2f}%")

Plate 1: exact_match=False, sequence_pct=12.5%
Plate 10: exact_match=False, sequence_pct=14.3%
Plate 100: exact_match=False, sequence_pct=25.0%
Plate 101: exact_match=False, sequence_pct=0.0%
Plate 102: exact_match=False, sequence_pct=25.0%
Plate 103: exact_match=False, sequence_pct=12.5%
Plate 104: exact_match=False, sequence_pct=62.5%
Plate 105: exact_match=False, sequence_pct=62.5%
Plate 106: exact_match=False, sequence_pct=12.5%
Plate 107: exact_match=False, sequence_pct=62.5%
Plate 108: exact_match=False, sequence_pct=25.0%
Plate 109: exact_match=False, sequence_pct=12.5%
Plate 11: exact_match=False, sequence_pct=0.0%
Plate 110: exact_match=False, sequence_pct=12.5%
Plate 111: exact_match=False, sequence_pct=87.5%
Plate 112: exact_match=False, sequence_pct=25.0%
Plate 113: exact_match=False, sequence_pct=62.5%
Plate 114: exact_match=False, sequence_pct=0.0%
Plate 115: exact_match=False, sequence_pct=0.0%
Plate 116: exact_match=False, sequence_pct=50.0%
Plate 117: exact_match=False

In [2]:
# Phase 5: Evaluate character recognition performance with sequence percentages
import os
import re
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Paths
GROUND_TRUTH_CSV = r"plate_labels.txt"     # 'name,plate' where plate is e.g. "6-jim,9-kh,…"
PREDICTION_DIR   = r"plate_results"        # contains <base>_plate_result.txt
OUTPUT_DIR       = r"evaluation_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def strip_prefix(token):
    return token.split('-', 1)[-1]

# 1. Load ground truth (strip numeric prefixes)
gt_map = {}
with open(GROUND_TRUTH_CSV, 'r', encoding='utf-8') as f:
    lines = [ln.strip() for ln in f if ln.strip()]
for ln in lines[1:]:
    parts = ln.split(',')
    if len(parts) < 2:
        continue
    name = parts[0].strip()
    tail = ','.join(parts[1:]).strip()
    raw_tokens = [t.strip() for t in tail.split(',') if t.strip()]
    gt_map[name] = [strip_prefix(t) for t in raw_tokens]

# 2. Iterate plates and compute metrics
plate_total = 0
plate_exact_matches = 0
sequence_scores = []       # per-plate % correct in sequence
all_gt, all_pred = [], []

for name, gt_labels in gt_map.items():
    base = name.split('_')[0]
    res_file = os.path.join(PREDICTION_DIR, f"{base}_plate_result.txt")
    if not os.path.isfile(res_file):
        print(f"Missing result for {name}")
        continue

    with open(res_file, 'r', encoding='utf-8') as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    final_line = next((ln for ln in lines if ln.startswith('FINAL:')), None)
    if not final_line:
        print(f"No FINAL line in {res_file}")
        continue

    raw_pred = re.findall(r'\d+-[^\d,]+', final_line)
    pred_labels = [strip_prefix(t) for t in raw_pred]

    L = min(len(gt_labels), len(pred_labels))
    if L == 0:
        print(f"Empty GT or prediction for {name}")
        continue

    gt_seq   = gt_labels[:L]
    pred_seq = pred_labels[:L]

    # 2a) plate‐level exact match
    plate_total += 1
    if gt_seq == pred_seq and len(gt_labels) == len(pred_labels):
        plate_exact_matches += 1

    # 2b) sequence percentage for this plate
    correct_chars = sum(1 for i in range(L) if gt_seq[i] == pred_seq[i])
    seq_pct = correct_chars / len(gt_labels) * 100
    sequence_scores.append(seq_pct)

    # accumulate for token‐level metrics
    all_gt.extend(gt_seq)
    all_pred.extend(pred_seq)

    print(f"Plate {name}: exact_match={gt_seq==pred_seq}, sequence_pct={seq_pct:.1f}%")

# 3. Aggregate
plate_exact_acc = plate_exact_matches / plate_total * 100 if plate_total else 0
avg_sequence_acc = np.mean(sequence_scores) if sequence_scores else 0
token_acc = accuracy_score(all_gt, all_pred) * 100 if all_gt else 0

# 4. Save & print
with open(os.path.join(OUTPUT_DIR, 'metrics.txt'), 'w', encoding='utf-8') as f:
    f.write(f"Plates processed: {plate_total}\n")
    f.write(f"Plate exact-match accuracy: {plate_exact_acc:.2f}%\n")
    f.write(f"Average sequence accuracy:   {avg_sequence_acc:.2f}%\n")
    f.write(f"Token-level accuracy:        {token_acc:.2f}%\n\n")
    if all_gt:
        f.write("Classification Report (token-level):\n")
        f.write(classification_report(all_gt, all_pred, zero_division=0))

print("\nPhase 5 evaluation complete.")
print(f"Plate exact-match: {plate_exact_acc:.2f}%")
print(f"Avg sequence acc:  {avg_sequence_acc:.2f}%")
print(f"Token-level acc:   {token_acc:.2f}%")


Plate 1: exact_match=False, sequence_pct=12.5%
Plate 10: exact_match=False, sequence_pct=14.3%
Plate 100: exact_match=False, sequence_pct=25.0%
Plate 101: exact_match=False, sequence_pct=0.0%
Plate 102: exact_match=False, sequence_pct=25.0%
Plate 103: exact_match=False, sequence_pct=12.5%
Plate 104: exact_match=False, sequence_pct=62.5%
Plate 105: exact_match=False, sequence_pct=62.5%
Plate 106: exact_match=False, sequence_pct=12.5%
Plate 107: exact_match=False, sequence_pct=62.5%
Plate 108: exact_match=False, sequence_pct=25.0%
Plate 109: exact_match=False, sequence_pct=12.5%
Plate 11: exact_match=False, sequence_pct=0.0%
Plate 110: exact_match=False, sequence_pct=12.5%
Plate 111: exact_match=False, sequence_pct=87.5%
Plate 112: exact_match=False, sequence_pct=25.0%
Plate 113: exact_match=False, sequence_pct=62.5%
Plate 114: exact_match=False, sequence_pct=0.0%
Plate 115: exact_match=False, sequence_pct=0.0%
Plate 116: exact_match=False, sequence_pct=50.0%
Plate 117: exact_match=False

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import ConfusionMatrixDisplay

# if all_gt:
#     cm_display = ConfusionMatrixDisplay.from_predictions(all_gt, all_pred, xticks_rotation=90)
#     plt.title("Confusion Matrix (Token Level)")
#     plt.savefig(os.path.join(OUTPUT_DIR, "confusion_matrix.png"))
#     plt.close()